THIS NOTEBOOK USING R AS ITS RUNTIME TYPE

In [1]:
## import data
weather <- read.csv('https://raw.githubusercontent.com/Zufaruu/Weather_Prediction/main/weatherAUS.csv')

In [2]:
## Importing & Installing Library
install.packages('caTools') #Train-Test-Split
library(caTools)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’




In [20]:
## Data Preprocessing
weather_cleaned = (weather[(!(is.na(weather$MinTemp) | weather$MinTemp=="")) & 
          (!(is.na(weather$MaxTemp) | weather$MaxTemp=="")) &
          (!(is.na(weather$Rainfall) | weather$Rainfall=="")) &  
          (!(is.na(weather$Evaporation) | weather$Evaporation=="")) &
          (!(is.na(weather$Sunshine) | weather$Sunshine=="")) &
          (!(is.na(weather$WindGustDir) | weather$WindGustDir=="")) &
          (!(is.na(weather$WindGustSpeed) | weather$WindGustSpeed=="")) &
          (!(is.na(weather$WindDir9am) | weather$WindDir9am=="")) &
          (!(is.na(weather$WindDir3pm) | weather$WindDir3pm=="")) &
          (!(is.na(weather$WindSpeed9am) | weather$WindSpeed9am=="")) &
          (!(is.na(weather$WindSpeed3pm) | weather$WindSpeed3pm=="")) &
          (!(is.na(weather$Humidity9am) | weather$Humidity9am=="")) &
          (!(is.na(weather$Humidity3pm) | weather$Humidity3pm=="")) &
          (!(is.na(weather$Pressure9am) | weather$Pressure9am=="")) &
          (!(is.na(weather$Pressure3pm) | weather$Pressure3pm=="")) &
          (!(is.na(weather$Cloud9am) | weather$Cloud9am=="")) &
          (!(is.na(weather$Cloud3pm) | weather$Cloud3pm=="")) &
          (!(is.na(weather$Temp9am) | weather$Temp9am=="")) &
          (!(is.na(weather$Temp3pm) | weather$Temp3pm=="")) &
          (!(is.na(weather$RainTomorrow) | weather$RainTomorrow==""))
        , ])
weather_cleaned['RainToday'][weather_cleaned['RainToday'] == 'Yes'] = 1
weather_cleaned['RainToday'][weather_cleaned['RainToday'] == 'No'] = 0
weather_cleaned['RainTomorrow'][weather_cleaned['RainTomorrow'] == 'Yes'] = 1
weather_cleaned['RainTomorrow'][weather_cleaned['RainTomorrow'] == 'No'] = 0
weather_cleaned$RainToday = as.numeric(weather_cleaned$RainToday)
weather_cleaned$RainTomorrow = as.numeric(weather_cleaned$RainTomorrow)

In [21]:
## Train-Test Split
set.seed(69)
split = sample.split(weather_cleaned,SplitRatio = 0.8)
trainc = subset(weather_cleaned,split == TRUE)
testc = subset(weather_cleaned,split == FALSE)

In [31]:
## Preprocess Train & Test Dataset 
trainc_tmr = subset(trainc, select = -c(Date))
testc_tmr= subset(testc, select = -c(Date))

In [32]:
## Doing Logistic Regression
lgr_Rain_Tomorrow = glm(RainTomorrow~., data = trainc_tmr, family = "binomial")
summary(lgr_Rain_Tomorrow)


Call:
glm(formula = RainTomorrow ~ ., family = "binomial", data = trainc_tmr)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.0049  -0.4989  -0.2696  -0.1101   3.2712  

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)              54.1562456  3.0957800  17.494  < 2e-16 ***
LocationBrisbane          0.7281660  0.1381997   5.269 1.37e-07 ***
LocationCairns            0.4194382  0.1434047   2.925 0.003446 ** 
LocationCanberra          0.2453685  0.1676143   1.464 0.143225    
LocationCobar             0.1870242  0.2377359   0.787 0.431464    
LocationCoffsHarbour      0.3481617  0.1504966   2.313 0.020700 *  
LocationDarwin           -0.0458666  0.1437791  -0.319 0.749721    
LocationHobart           -0.2994260  0.1496817  -2.000 0.045455 *  
LocationMelbourne         0.1324660  0.1463504   0.905 0.365397    
LocationMelbourneAirport -0.1520433  0.1418784  -1.072 0.283879    
LocationMildura           0.2149873  0.15258

In [37]:
## Predicting
predict_tmr = predict(lgr_Rain_Tomorrow, testc_tmr, type="response")
predict_tmr = ifelse(predict_tmr > 0.7, 1, 0)
table(testc_tmr$RainTomorrow, predict_tmr)

   predict_tmr
       0    1
  0 9339  186
  1 1744  996

In [50]:
## Accuracy
paste(as.character( ( (9339+996)*100 / (9339+996+1744+186) )), "%", sep=" ") 

[1] "84.2641663269466 %"